# 01 Import Libraries

In [1]:
import os
import numpy as np
import pandas as pd

# 02 Import Data

In [2]:
path = r'/Users/carlo/21-06-2022 Instacart Basket Analysis'

In [3]:
df_orders_products_merged = pd.read_pickle(os.path.join(path, '2. Data', 'Prepared Data', 'orders_products_altered.pkl'))

# 03 Agreggating Data

In [4]:
#Grouping data
df_orders_products_merged.groupby('product_name')

In [5]:
#aggregating average orders per department
df_orders_products_merged.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


# 04 creating a loyaty flag

In [6]:
#Aggregating orders for customers
df_orders_products_merged['max_order'] = df_orders_products_merged.groupby(['user_id'])['order_number'].transform(np.max)

In [7]:
pd.options.display.max_rows = None

In [8]:
df_orders_products_merged.head()

,Unnamed: 0_x,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,...,product_name,aisle_id,department_id,prices,_merge,price_range,busiest_day,busiest_days,busiest_period_of_day,max_order
0,0,2539329,1,1,2,8,NaN,196.0,1.0,0.0,...,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10
1,1,2398795,1,2,3,7,15.0,196.0,1.0,1.0,...,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy days,Average orders,10
2,2,473747,1,3,3,12,21.0,196.0,1.0,1.0,...,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy days,Most orders,10
3,3,2254736,1,4,4,7,29.0,196.0,1.0,1.0,...,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy days,Average orders,10
4,4,431534,1,5,4,15,28.0,196.0,1.0,1.0,...,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy days,Most orders,10


In [9]:
# using the loc() function to group the customers
df_orders_products_merged.loc[df_orders_products_merged['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
df_orders_products_merged.loc[(df_orders_products_merged['max_order'] <= 40) & (df_orders_products_merged['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
df_orders_products_merged.loc[df_orders_products_merged['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [10]:
df_orders_products_merged['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

In [11]:
#check the average price of product purchase by the customer groups
df_orders_products_merged.groupby('loyalty_flag').agg({'prices':['mean','max','min']})

prices           
                      mean   max  min
loyalty_flag                         
Loyal customer    7.773575  25.0  1.0
New customer      7.801206  25.0  1.0
Regular customer  7.798262  25.0  1.0

# 05. Creating the spender Flag

In [12]:
#Aggregating spending for customers
df_orders_products_merged['mean_spending'] = df_orders_products_merged.groupby(['user_id'])['prices'].transform(np.mean)

In [13]:
df_orders_products_merged.head()

,Unnamed: 0_x,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,...,department_id,prices,_merge,price_range,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_spending
0,0,2539329,1,1,2,8,NaN,196.0,1.0,0.0,...,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797
1,1,2398795,1,2,3,7,15.0,196.0,1.0,1.0,...,7,9.0,both,Mid-range product,Regularly busy,Least busy days,Average orders,10,New customer,6.367797
2,2,473747,1,3,3,12,21.0,196.0,1.0,1.0,...,7,9.0,both,Mid-range product,Regularly busy,Least busy days,Most orders,10,New customer,6.367797
3,3,2254736,1,4,4,7,29.0,196.0,1.0,1.0,...,7,9.0,both,Mid-range product,Least busy,Least busy days,Average orders,10,New customer,6.367797
4,4,431534,1,5,4,15,28.0,196.0,1.0,1.0,...,7,9.0,both,Mid-range product,Least busy,Least busy days,Most orders,10,New customer,6.367797


In [14]:
#deriving new column with loc
df_orders_products_merged.loc[df_orders_products_merged['mean_spending'] >= 10, 'spender_flag'] = 'High spender'
df_orders_products_merged.loc[df_orders_products_merged['mean_spending'] < 10, 'spender_flag'] = 'Low spender'

In [15]:
df_orders_products_merged['spender_flag'].value_counts(dropna = False)

Low spender     32285131
High spender      119728
Name: spender_flag, dtype: int64

# 06. Creating the Frequency Flag

In [16]:
#create a new column according to mean since last order
#Aggregating spending for customers
df_orders_products_merged['order_frequency'] = df_orders_products_merged.groupby(['user_id'])['days_since_last_order'].transform(np.median)

In [17]:
df_orders_products_merged.head()

,Unnamed: 0_x,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,...,_merge,price_range,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_spending,spender_flag,order_frequency
0,0,2539329,1,1,2,8,NaN,196.0,1.0,0.0,...,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5
1,1,2398795,1,2,3,7,15.0,196.0,1.0,1.0,...,both,Mid-range product,Regularly busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,20.5
2,2,473747,1,3,3,12,21.0,196.0,1.0,1.0,...,both,Mid-range product,Regularly busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,20.5
3,3,2254736,1,4,4,7,29.0,196.0,1.0,1.0,...,both,Mid-range product,Least busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,20.5
4,4,431534,1,5,4,15,28.0,196.0,1.0,1.0,...,both,Mid-range product,Least busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,20.5


In [18]:
#deriving new column with loc
#Adding frequency flags
df_orders_products_merged.loc[df_orders_products_merged['order_frequency'] > 20, 'frequency_flag'] = 'Non-frequent customer'
df_orders_products_merged.loc[(df_orders_products_merged['order_frequency'] <= 20) & (df_orders_products_merged['order_frequency'] > 10), 'frequency_flag'] = 'Regular customer'
df_orders_products_merged.loc[df_orders_products_merged['order_frequency'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [19]:
df_orders_products_merged['frequency_flag'].value_counts(dropna = False)

Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: frequency_flag, dtype: int64

# 07 Export Data

In [20]:
# Export data to pkl
df_orders_products_merged.to_pickle(os.path.join(path, '2. Data','Prepared Data', 'orders_products_grouped.pkl'))

In [21]:
df_orders_products_merged.groupby('orders_day_of_week').agg({'prices': ['mean']})

,prices
,mean
orders_day_of_week,
0,7.857590
1,7.759697
2,7.754309
3,7.757485
4,7.756223
5,7.762477
6,7.857431
